# 分类

In [1]:
# KNN 
# 朴素贝叶斯
# 决策树
# 支持向量机
# 集成方法
# 罗吉斯特映射（以回归为主，可做分类，可做回归）
# 人工神经网络（以回归为主，可做分类，可做回归）

In [2]:
# 距离
# 欧式距离：两个点的直线距离
# 曼哈顿距离：各个维度的长度差的绝对值的和（横坐标的长度差加上纵坐标的长度差）
# 闵可夫斯基距离：各个维度的长度差的绝对值p次方的和，开p次根号
# KD-Tree
# 如果一个空间中有很多的点，那么如何找到我们随机指定的一个点的附近的K个点。
# 通过一个维度将所有的点分成两个部分，两部分的数量尽可能的保持一致。
# 然后拿其中的一部分再进行另一个维度上的切分，一次类推，直到不能切分了为止
# 这样的话，在空间中就建立了许多大小不一的格子，每个点通过格子的线进行链接（线当作是中间节点，点当作叶子节点），构成了一个树形结构。

### KNN （K-Nearest Neighbors）

In [3]:
# 算法思想
# 一个数据集都会有它的标注，那么如果找到了一个点，它k个最近的邻居，一种标注大于了另外一种标注，那么我们就认为，这个点更倾向于与多数点是一致的。

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA

In [29]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)
        
    
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    # print(len(X_train), len(X_validation), len(X_test))
    
    # 衡量指标
    # accuracy_score 准确率
    # recall_score 召回率
    # f1_score F值
    from sklearn.metrics import accuracy_score, recall_score, f1_score
    # KNN
    # NearestNeighbors 此方法可以直接获得一个点附近最近的几个点
    from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
    models = []
    # n_neigbors 获取最近的几个点
    # radius 半径
    # algorithm:{'auto', 'ball_tree', 'kd_tree', 'brute'} 指定索引方法
    # kd_tree 针对方格子进行的索引  ball_tree 针对球体进行的索引  brute 遍历
    # p 就是闵可夫斯基距离公式中的 p 默认为2 
    # n_jobs 并行计算的单元数量
    knn_clf = KNeighborsClassifier(n_neighbors=3)
    knn_clf.fit(X_train, Y_train)
        
    # 利用训练集进行预测
    print("Train")
    Y_pred = knn_clf.predict(X_train)
    print("ACC:", accuracy_score(Y_train, Y_pred))
    print("REC:", recall_score(Y_train, Y_pred))
    print("F-Score:", f1_score(Y_train, Y_pred))
    
    print("Validation")
    # 预测 验证集 得到一个预测值
    Y_pred = knn_clf.predict(X_validation)
    # 拿预测值与实际值进行比较
    print("ACC:", accuracy_score(Y_validation, Y_pred))
    print("REC:", recall_score(Y_validation, Y_pred))
    print("F-Score:", f1_score(Y_validation, Y_pred))
    
    # 利用测试集进行对比
    print("Test")
    Y_pred = knn_clf.predict(X_test)
    print("ACC_test:", accuracy_score(Y_test, Y_pred))
    print("REC_test:", recall_score(Y_test, Y_pred))
    print("F-Score_test:", f1_score(Y_test, Y_pred))
    
    # 将模型进行存储
    from sklearn.externals import joblib
    # 保存模型
    # joblib.dump(knn_clf, "knn_clf")
    # 使用模型
    knn_clf = joblib.load("knn_clf")
    print("Test_2")
    Y_pred = knn_clf.predict(X_test)
    print("ACC_test:", accuracy_score(Y_test, Y_pred))
    print("REC_test:", recall_score(Y_test, Y_pred))
    print("F-Score_test:", f1_score(Y_test, Y_pred))
    
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    hr_modeling(features, label)
if __name__=="__main__":
    main()

Train
ACC: 0.9742193577064118
REC: 0.959794296400187
F-Score: 0.9465191332411249
Validation
ACC: 0.9546666666666667
REC: 0.9364406779661016
F-Score: 0.9069767441860463
Test
ACC_test: 0.959
REC_test: 0.9281767955801105
F-Score_test: 0.9161554192229038
Test_2
ACC_test: 0.9703333333333334
REC_test: 0.9502762430939227
F-Score_test: 0.9392491467576791


### 朴素贝叶斯 

In [26]:
# 概率：P(A)
# 条件概率：P(A|B) 条件B发生的条件下，条件A发生的概率
# 联合概率：P(A, B) 条件A,B同时发生的概率 
# P(A, B) = P(A|B)P(B) = P(B|A)P(A)
# 全概率公式：B的概率等于：A事件发生的各种情况下，B事件发生的概率的和
# 拉普拉斯平滑：若存在0值，就将所有的数据都加上1，可以规避样本数量等于0的情况

In [15]:
# 归一化将数据转化到 0-1 之间的范围
# sl:satisfaction_level
# le:last_evaluation
# npr:number_project
# amh:average_monthly_hours
# tsc:time_spend_company
# wa:Work_accident
# pl5:promotion_last_5years
# False:MinMaxScaler
# True:StandardScaler
# 将离散值数值化
# dp:department 
# slr:salary
# False:LabelEncoding
# True:OneHotEncoder
# 降维
# lower_d = False  不降维
# ld_n = 1 下降的维度
def hr_preprocessing(sl=False, le=False, npr=False, amh=False, tsc=False, wa=False, pl5=False, dp=False, slr=False, lower_d = False, ld_n = 1):
    df = pd.read_csv('./data/HR.csv')
    # 1.清洗数据
    df = df.dropna(subset=['satisfaction_level', 'last_evaluation'])
    df = df[df['satisfaction_level']<=1][df['salary']!='nme']
    # 2.得到标注
    label = df["left"]
    df = df.drop('left', axis=1)
    # 3.特征选择
    
    # 4.特征处理
    scaler_lst = [sl, le, npr, amh, tsc, wa, pl5]
    column_lst = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_monthly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            df[column_lst[i]]=StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
    # # 将离散值数值化
    scaler_lst = [dp, slr]
    column_lst = ["department", "salary"]
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i] == "salary":
                df[column_lst[i]] = [map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]] = LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]] = MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1, 1)).reshape(1, -1)[0]
        else:
            # OneHotEncoder
            df = pd.get_dummies(df, columns=[column_lst[i]])
    # 降维
    if lower_d:
        # n_components 不能大于类的个数
        # return LinearDiscriminantAnalysis(n_components=ld_n)
        # PCA n_components 不受限制
        return PCA(n_components=ld_n).fit_transform(df.values)  
    return df, label

d = dict([('low', 0), ('medium', 1), ('high', 2)])
def map_salary(s):
    return d.get(s, 0)

# 训练集 验证集 测试集
def hr_modeling(features, label):
    # 引入切分训练集和测试集的函数
    from sklearn.model_selection import train_test_split
    # 先取值
    f_v = features.values
    # 特征的名称
    f_names = features.columns.values
    # 特征的标签
    l_v = label.values
    # test_size 测试集占多少比例  
    # X_tt  训练集部分 X_validation标注
    # y_tt  验证集部分 Y_validation标注
    # 得到验证集
    X_tt, X_validation, Y_tt, Y_validation = train_test_split(f_v, l_v, test_size=0.2)
    # 区分验证集和测试集
    X_train, X_test, Y_train, Y_test = train_test_split(X_tt, Y_tt, test_size=0.25)
    # print(len(X_train), len(X_validation), len(X_test))
    
    # 衡量指标
    # accuracy_score 准确率
    # recall_score 召回率
    # f1_score F值
    from sklearn.metrics import accuracy_score, recall_score, f1_score
    # KNN
    # NearestNeighbors 此方法可以直接获得一个点附近最近的几个点
    from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
    # 朴素贝叶斯 （适用与离散型的数值）
    # GaussianNB 高斯朴素贝叶斯 （特征是高斯分布的）
    # BernoulliNB伯努利朴素贝叶斯 （数值是离散的，二值，0和1）若是连续值，也需要先将数值离散化
    from sklearn.naive_bayes import GaussianNB, BernoulliNB
    # 决策树
    from sklearn.tree import DecisionTreeClassifier,export_graphviz
    from sklearn.externals.six import StringIO
    # 引入SVM  分类器
    from sklearn.svm import SVC
    # 随机森林
    from sklearn.ensemble import RandomForestClassifier
    # Adaboost
    from sklearn.ensemble import AdaBoostClassifier
    
    # 图形展示
    import pydotplus
    
    models = []
    # n_neigbors 获取最近的几个点
    # radius 半径
    # algorithm:{'auto', 'ball_tree', 'kd_tree', 'brute'} 指定索引方法
    # kd_tree 针对方格子进行的索引  ball_tree 针对球体进行的索引  brute 遍历
    # p 就是闵可夫斯基距离公式中的 p 默认为2 
    # n_jobs 并行计算的单元数量
    models.append(("KNN", KNeighborsClassifier(n_neighbors=3)))
    # 高斯贝叶斯
    models.append(("GaussianNB:", GaussianNB()))
    # 伯努利朴素贝叶斯
    models.append(("BernoulliNB:", BernoulliNB()))
    # Gini决策树
    # min_impurity_split=0.1 最小不纯度的切分
#     models.append(("DecisionTreeGini",DecisionTreeClassifier(min_impurity_split=0.1)))
    models.append(("DecisionTreeGini",DecisionTreeClassifier()))
    # 信息增益决策树
    models.append(("DecisionTreeEntropy",DecisionTreeClassifier(criterion="entropy")))
    # SVM  分类器
    # kernel 核函数 取值 线性 linear、多项式 poly、高斯定向集 rbf、
    # degree 如果我们核函数选了多项式，那么我们选择几阶
    # max_iter 最大迭代次数
    # tol 精度
    # decision_function_shape 用于多分类，ovo ovr  默认ovr
    # C 如果一个标注分错的话会有多大的惩罚,值越大，就会让更多的点不被错分
    models.append(("SVM Classifier", SVC(C = 1000)))
    # 添加随机森林
    # n_estimators 决策树的个数  默认10棵
    # criterion 使用的方法  默认是Gini
    # max_features 每棵树用了多少特征 默认是auto = sqrt 可选值有 int, float（比例）, auto, sqrt, log2, None(代表全量特征)
    # max_depth 最大深度
    # min_samples_split 最小样本切分
    # min_samples_leaf 一个叶子中的最小样本数
    # bootstrap 是否进行有放回的采样 True是  False不放回
    # oob_score 在使用放回采样的基础上，使用没有被采集到的样本去评估整体的准确性
    # n_jobs 并行数量
    models.append(("RandomForest", RandomForestClassifier(max_features=None, n_estimators=10)))
    # Adaboost
    # random_state 随机数的种子值
    # base_estimator 弱分类的基本分类器 需要两个属性 classes_ 和 n_classes_ 默认的是决策树
    # n_estimators 及联的决策树的数量 默认是50个
    # learning_rate 根据0-1的数进行衰减
    # algorithm 针对 base_estimator 进行的选择，如果base_estimator可以得到一个判别的概率，那么就可以使用SAMME.R,得到更好的效果
    models.append(("AdaBoost", AdaBoostClassifier(n_estimators=100)))
    
    for clf_name, clf in models:
        # 先进行拟合 训练
        clf.fit(X_train, Y_train)
        # 进行预测和评价 【训练集，验证集，测试集】
        xy_lst = [(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)]
        for i in range(len(xy_lst)):
            # X_part
            X_part = xy_lst[i][0]
            # y_part
            Y_part = xy_lst[i][1]
            # 预测值
            Y_pred = clf.predict(X_part)
            # 0训练集 1验证集 2测试集
            print(i)
            print(clf_name, "-ACC:", accuracy_score(Y_part,  Y_pred))
            print(clf_name, "-REC:", recall_score(Y_part,  Y_pred))
            print(clf_name, "-F-Score:", f1_score(Y_part,  Y_pred))
            # 使用 graphviz 将决策树以图形的方式展示出来
            # out_file=None 可以直接输出
            # feature_names 特征的名称
            # class_names 标注的名称
            # filled=True 填充
            # 方法一：
#             dot_data = export_graphviz(clf, out_file=None, 
#                             feature_names=f_names,
#                             class_names=["NL", "L"], 
#                             filled=True,
#                             rounded=True,
#                             special_characters=True
#                            )
#             # 绘制图形
#             graph = pydotplus.graph_from_dot_data(dot_data)
#             # 输出为PDF格式
#             graph.write_pdf("dt_tree.pdf")

#             # 方法二：
#             dot_data = StringIO()
#             export_graphviz(clf, out_file=dot_data, 
#                             feature_names=f_names,
#                             class_names=["NL", "L"], 
#                             filled=True,
#                             rounded=True,
#                             special_characters=True
#                            )
#             # 绘制图形
#             graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
#             # 输出为PDF格式
#             graph.write_pdf("dt_tree_2.pdf")

    
def main():
    # print(hr_preprocessing(lower_d = False, ld_n = 3))
    features, label = hr_preprocessing()
    hr_modeling(features, label)
if __name__=="__main__":
    main()

0
KNN -ACC: 0.9742193577064118
KNN -REC: 0.9581993569131833
KNN -F-Score: 0.9473206176203451
1
KNN -ACC: 0.953
KNN -REC: 0.9184247538677919
KNN -F-Score: 0.902557014512785
2
KNN -ACC: 0.9546666666666667
KNN -REC: 0.9326500732064422
KNN -F-Score: 0.9035460992907802
0
GaussianNB: -ACC: 0.7960884542726969
GaussianNB: -REC: 0.7322002756086358
GaussianNB: -F-Score: 0.6346804698387418
1
GaussianNB: -ACC: 0.7796666666666666
GaussianNB: -REC: 0.7285513361462729
GaussianNB: -F-Score: 0.610489098408957
2
GaussianNB: -ACC: 0.7986666666666666
GaussianNB: -REC: 0.7423133235724744
GaussianNB: -F-Score: 0.626699629171817
0
BernoulliNB: -ACC: 0.8409823313701522
BernoulliNB: -REC: 0.4694533762057878
BernoulliNB: -F-Score: 0.5882014388489208
1
BernoulliNB: -ACC: 0.8373333333333334
BernoulliNB: -REC: 0.46272855133614627
BernoulliNB: -F-Score: 0.5741710296684118
2
BernoulliNB: -ACC: 0.8476666666666667
BernoulliNB: -REC: 0.47584187408491946
BernoulliNB: -F-Score: 0.5871725383920505
0
DecisionTreeGini -ACC:

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0
SVM Classifier -ACC: 0.9603289254361596
SVM Classifier -REC: 0.9076711070280202
SVM Classifier -F-Score: 0.9171501508470643
1
SVM Classifier -ACC: 0.9566666666666667
SVM Classifier -REC: 0.9057665260196905
SVM Classifier -F-Score: 0.9083215796897037
2
SVM Classifier -ACC: 0.961
SVM Classifier -REC: 0.9297218155197657
SVM Classifier -F-Score: 0.9156452775775054
0
RandomForest -ACC: 0.9984442715857318
RandomForest -REC: 0.994487827285255
RandomForest -F-Score: 0.9967771639042357
1
RandomForest -ACC: 0.986
RandomForest -REC: 0.9648382559774965
RandomForest -F-Score: 0.9702970297029703
2
RandomForest -ACC: 0.9913333333333333
RandomForest -REC: 0.9736456808199122
RandomForest -F-Score: 0.9808259587020649
0
AdaBoost -ACC: 0.9604400488943216
AdaBoost -REC: 0.9131832797427653
AdaBoost -F-Score: 0.917820867959372
1
AdaBoost -ACC: 0.9563333333333334
AdaBoost -REC: 0.9043600562587905
AdaBoost -F-Score: 0.9075511644318985
2
AdaBoost -ACC: 0.9666666666666667
AdaBoost -REC: 0.9282576866764275
AdaB

### 生成模型与判别模型

In [1]:
# 生成模型（对数据要求高，速度快一些）：通过求输入与输出的联合概率分布，再求解类别归类的概率。比如朴素贝叶斯模型（因为其分子是一个联合概率分布）
# 判别模型（数据要求不高，速度慢一些，使用的范围更广）：不通过联合概率分布，直接就可以获得输出对应最大分类的概率。比如KNN

### 决策树

In [2]:
# 决策树就是模仿我们做决策的过程，一步步按照特征进行判断。 例如相亲图
# 如何决定特征的顺序
# 1.信息增益-ID3:回顾熵（是随机变量或整个系统的不确定性，熵越大，随机变量的不确定性就越大）
    # 熵变化的大小   每次都选择熵增益大的
# 2.信息增益率-C4.5
    # 分子是信息增益I(X,Y)，分母是Y的熵H(Y)
# 3.Gini系数-CART  也叫不纯度
    # 用1减去各个因子的数量除以它的总数再平方，得到一个不纯度。如果一个切分，它的不纯度减小的比较大，我们就可以考虑将这个切分先进行决策。
    # 不纯度最低的切分作为当前的切分
# ⚠️需要注意的问题
# 1.连续值的切分
    # 将连续值进行从小到大的排序，然后对每个间隔进行切分，计算切分后的各个因子，取该因子性能最好的连续值切分，作为它的切分。
# 2.规则用尽
    # 比如有四个特征。有可能四个特征都用完了，但是所剩的数据集合还没有切分干净，这样我们可以采用投票的方式，就是哪个样本多就投哪个，当然也可以接着使用特征进行切分，也就是一个特征可以用多次，进行进一步的切分，最终得到没有杂志的叶子节点。
# 3.过拟合
    # 如果一个特征可以切割多次，那么最终我们总可以得到一个百分之百的切分方案，但是也会带来过拟合的问题。就需要考虑修枝剪叶。
    # 前剪枝：构造决策树之前，我就规定了每个叶子节点最多有多少个样本或者规定了这颗决策树的最大深度。
    # 后剪枝：先想尽一切办法构造出一颗决策树，然后对一些样本值比较悬殊的枝叶进行修剪
# mac下的 Graphviz 安装
# brew install graphviz

#### 代码 （看上面）

### SVM 支持向量机 

In [9]:
# 比如有两个标注的数据，假如想用一条线将其区分开，会有无数种分法。但只有一种分法可以将两个样本最大程度的区分，也就是说两个标注样本中，分别找出离这条线最近的点，它们离这条线的距离是一样的，并且距离和是最大的。这样的切分就是区分度最大的切分方法。此时离这条线最近的两个标注中的样本，就是SVM中的支持向量。
# 多维空间中的面也叫做超平面
# 高维面：wTx + b = 0  所得到的面就是分界面，所以一定会有一种标注的样本将值带入，一定大于0，另一种标注的样本值带入，一定都小于0.考虑到多数情况下，样本点不会落在直线上，而与这条直线会有一段距离，所以我们可以假设这段距离。
# 分界面: wTxp + b >= e  wTxn + b <= -e 
# 可简化为：wTxp + b >= 1  wTxn + b <= -1
# 遇到复杂情况有两种解决情况
# 1.容忍一部分错误情况
# 2.扩维：先计算（低维空间），再扩维（利用核函数扩维）
# 常用的核函数
    # 1.线性核函数：在线性可分的情况下使用
    # 2.多项式核函数：二维数据d取2就可以扩充到5维
    # 3.高斯颈向基（RBF）核函数：可以将空间映射为无限维
# 相对于决策树，SVM的边界更加平滑
# ⚠️
# 少部分异常：引入松弛变量（在原来的变量中再加入一个变量），为了达到更大的平衡，可容忍少量的错分点。
# 样本不平衡：看场景定方案
# 多分类：
    # one-other：有几个分类就建几个SVM，一个样本过来，我们把每个SVM都过一遍，找成功分类并且离超平面距离最远的一个作为正确分类。
    # one-one：分类的两两之间分别进行SVM，比如有4个分类，就会有6个SVM。等样本过来的时候，两两间进行比较进行分类，取出一个被分类次数最多的那个类作为我们的SVM分类。

#### 代码演示 （看上面）

### 集成方法-随机森林

In [1]:
# 回顾：熵增益的决策树适合解决离散值比较多的问题。而基于Gini系数-CART 决策树适合解决连续值分类的问题。如果标注在空间内，隔离性比较好，KNN、SVM较合适。
# 分类器就是一种算法。
# 集成学习：就是组合多个模型，以获得更好的效果。
# 强可学习：多项式学习算法的效果较为明显
# 弱可学习：多项式学习算法的效果不是很明显
# 模型的集成方法就可以看做是将多个弱可学习分类器，集合成一个强可学习的分类器的过程。
# 集成思想：
    # 1.袋装法（bagging）：使用训练集同时训练出几个独立的模型，而在进行预测和判断时，我们分别让被训练出的几个子模型去进行判断，然后对于分类问题我们让它们去投票，它们投票选出最多的结果就是最终判断的结果。
    # 袋装法的典型应用：随机森林（需要注意的地方：树的个数，树的特征数（<50即可采用全特征）。
    # 树的训练集：一般都是模型训练集的一个子集，如何选择子集：我们可以把选择子集的数量定为和输入的模型的数量是一致的，只是对其进行有放回的采样，大概率的有些样本被采集2次以上，有些一次都没被采集到。通过样本构成了树的差异性。）
    # 随机森林使用决策树的好处：1.每个决策树可以并不使用全部的特征。这样减少了模型的规模和计算的复杂度，也可以得到不错的效果。2.不需要剪枝，即可有效避免过拟合 
    # 2.提升法（）：

#### 代码实现 看上面

### 集成方法-提升法（boost）

In [9]:
# 提升法：把子模型串联起来，一个模型以另一个模型的结果为基础进行训练和预测，然后多个模型及联，最终将每个模型的结果进行加权求和得到判决结果
# Adaboost 例子
    # 优点：
    # 精度高，且灵活可调
    # 几乎不用担心过拟合
    # 简化特征工程流程

#### 代码实现 （看上面）